In [1]:
import os
from pathlib import Path
compe='commonlit-evaluate-student-summaries'
is_kaggle=os.environ.get('KAGGLE_KERNEL_RUNTYPE','')
if is_kaggle:
    path=Path(f'../input/{compe}')
    ! pip install -Uqq fastai
else:
    import kaggle,zipfile
    path=Path(f'{compe}')
    if not path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{compe}.zip').extractall(path)

100%|██████████████████████████████████████| 1.05M/1.05M [00:00<00:00, 6.68MB/s]

In [2]:
from fastai.tabular.all import *
from sklearn.metrics import roc_auc_score

pd.options.display.float_format = '{:.2f}'.format
set_seed(42)

In [5]:
!cd commonlit-evaluate-student-summaries;ls

prompts_test.csv      sample_submission.csv summaries_train.csv
prompts_train.csv     summaries_test.csv


In [16]:
df=pd.read_csv(f'{compe}/prompts_train.csv')
df_test=pd.read_csv(f'{compe}/prompts_test.csv')
df_sub=pd.read_csv(f'{compe}/sample_submission.csv')
df_sum=pd.read_csv(f'{compe}/summaries_train.csv')
df_sum_test=pd.read_csv(f'{compe}/summaries_test.csv')

len(df),len(df_test)

(4, 2)

In [24]:
df.head(10)

,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,"Summarize at least 3 elements of an ideal tragedy, as described by Aristotle.",On Tragedy,"Chapter 13 \r\nAs the sequel to what has already been said, we must proceed to consider what the poet should aim at, and what he should avoid, in constructing his plots; and by what means the specific effect of Tragedy will be produced. \r\nA perfect tragedy should, as we have seen, be arranged not on the simple but on the complex plan. It should, moreover, imitate actions which excite pity and fear, this being the distinctive mark of tragic imitation. It follows plainly, in the first place, that the change of fortune presented must not be the spectacle of a virtuous man brought from prosp..."
1,3b9047,"In complete sentences, summarize the structure of the ancient Egyptian system of government. How were different social classes involved in this government? Cite evidence from the text.",Egyptian Social Structure,"Egyptian society was structured like a pyramid. At the top were the gods, such as Ra, Osiris, and Isis. Egyptians believed that the gods controlled the universe. Therefore, it was important to keep them happy. They could make the Nile overflow, cause famine, or even bring death. \r\nThe Egyptians also elevated some human beings to gods. Their leaders, called pharaohs, were believed to be gods in human form. They had absolute power over their subjects. After pharaohs died, huge stone pyramids were built as their tombs. Pharaohs were buried in chambers within the pyramids. \r\nBecause the pe..."
2,814d6b,Summarize how the Third Wave developed over such a short period of time and why the experiment was ended.,The Third Wave,"Background \r\nThe Third Wave experiment took place at Cubberley High School in Palo Alto, California during the first week of April 1967. History teacher Ron Jones, finding himself unable to explain to his students how people throughout history followed the crowd even when terrible things were happening, decided to demonstrate it to his students through an experiment. Jones announced that he was starting a movement aimed to eliminate democracy. Jones named the movement “The Third Wave” as a symbol of strength, referring to the mythical belief that the third in a series of waves is the str..."
3,ebad26,Summarize the various ways the factory would use or cover up spoiled meat. Cite evidence in your answer.,Excerpt from The Jungle,"With one member trimming beef in a cannery, and another working in a sausage factory, the family had a first-hand knowledge of the great majority of Packingtown swindles. For it was the custom, as they found, whenever meat was so spoiled that it could not be used for anything else, either to can it or else to chop it up into sausage. With what had been told them by Jonas, who had worked in the pickle rooms, they could now study the whole of the spoiled-meat industry on the inside, and read a new and grim meaning into that old Packingtown jest—that they use everything of the pig except the ..."


In [15]:
df.prompt_text.values[0]

'Chapter 13 \r\nAs the sequel to what has already been said, we must proceed to consider what the poet should aim at, and what he should avoid, in constructing his plots; and by what means the specific effect of Tragedy will be produced. \r\nA perfect tragedy should, as we have seen, be arranged not on the simple but on the complex plan. It should, moreover, imitate actions which excite pity and fear, this being the distinctive mark of tragic imitation. It follows plainly, in the first place, that the change of fortune presented must not be the spectacle of a virtuous man brought from prosperity to adversity: for this moves neither pity nor fear; it merely shocks us. Nor, again, that of a bad man passing from adversity to prosperity: for nothing can be more alien to the spirit of Tragedy; it possesses no single tragic quality; it neither satisfies the moral sense nor calls forth pity or fear. Nor, again, should the downfall of the utter villain be exhibited. A plot of this kind would, 

In [12]:
df_sum.head(10)

,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how people reacted to a new one leader government. It gained popularity as people wanted to try new things. The students follow anything that is said and start turning on eachother to gain higher power. They had to stop the experement as too many people got to radical with it blindly following there leader,0.21,0.38
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the smell go away and it wouldnt be a bad smell. Some of the meat would be tossed on the floor where there was sawdust spit of the workers and they would make the meat all over again with the things in it.,-0.55,0.51
2,004e978e639e,3b9047,"In Egypt, there were many occupations and social classes involved in day-to-day living. In many instances if you were at the bottom of the social ladder you could climb up, you didn't have to stay a peasant you could work to bring your status up. Everyone worshipped the gods Ra, Osiris, and Isis, but also they would worship their pharaohs like gods as well. Under the pharaohs were the priests, they had the responsibility to entertain or please the said god. The Chain of Command was placed to keep everyone in check, not one person could handle all the civilians and treasures without any aid...",3.13,4.23
3,005ab0199905,3b9047,The highest class was Pharaohs these people were gods.Then the 2nd highest class was a gonvener.Chiefs minister were called a vizier as a supervisor. (par.6),-0.21,-0.47
4,0070c9e7af47,814d6b,"The Third Wave developed rapidly because the students genuinly believed that it was the best course of action. Their grades, acomplishments, and classparticipation/ behavior had improved dramatically since the experiment began. There did not seem to be any consiquenses in the students eyes. They became extremely engaged in all the Third Wave activites both inside and outside tha classroom. The experiment ended because the students were so patriotic about the ""movement"". The history class of thirty rapidly grew to 200 in three days. That means 170 students joined a school ""movement"" in tw...",3.27,3.22
5,0071d51dab6d,ebad26,They would use chemicals and substances to change the color or hide the foul odors to cover up spoiled meat. They would also shred up spoiled and normal meat together into sausage and use many different substances to mask the signs of spoiling.,0.21,0.38
6,0072b649a88c,3b9047,The Egyptian society is really different from other socicties I have learned about. As stated in the story in paragraph 6-13 it describes how people are in different places in the pyramid. Also tells us how people are treated and what they have to pay for things and the higher people have things more easy.All the classes have a different purpose which are important in their different wyas. This is how social classes are involved in there government.,0.21,0.38
7,00746c7c79c3,ebad26,"Many times the factories would, according to the text, ¨rub it up with soda to take away the smell."" that is just one of the ways some of the aother ways were rubbing chemicals on the meat and cooking the meat until the rotted smell again.",-0.88,-0.97
8,00791789cc1f,39c16e,1 element of an ideal tragedy is that it should be arranged on a complex plan. Another element of an ideal tragedy is that it should only have one main issue. The last element of an ideal tragedy is that it should have a double thread plot and an opposite catastrophe for both good and bad.,-0.21,-0.47
9,0086ef22de8f,39c16e,"The three elements of an ideal tragedy are: Having a character that isn't bad have misfortune befall them., Having no subplots, and ending in death.",-0.97,-0.42


In [17]:
df_sum.shape,df_sum_test.shape

((7165, 5), (4, 3))

In [18]:
df_sum_test.head(10)

,student_id,prompt_id,text
0,000000ffffff,abc123,Example text 1
1,111111eeeeee,def789,Example text 2
2,222222cccccc,abc123,Example text 3
3,333333dddddd,def789,Example text 4


In [19]:
df_sub.head(10)

,student_id,content,wording
0,000000ffffff,0.00,0.00
1,111111eeeeee,0.00,0.00
2,222222cccccc,0.00,0.00
3,333333dddddd,0.00,0.00


In [20]:
df.head(10)

,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,"Summarize at least 3 elements of an ideal tragedy, as described by Aristotle.",On Tragedy,"Chapter 13 \r\nAs the sequel to what has already been said, we must proceed to consider what the poet should aim at, and what he should avoid, in constructing his plots; and by what means the specific effect of Tragedy will be produced. \r\nA perfect tragedy should, as we have seen, be arranged not on the simple but on the complex plan. It should, moreover, imitate actions which excite pity and fear, this being the distinctive mark of tragic imitation. It follows plainly, in the first place, that the change of fortune presented must not be the spectacle of a virtuous man brought from prosp..."
1,3b9047,"In complete sentences, summarize the structure of the ancient Egyptian system of government. How were different social classes involved in this government? Cite evidence from the text.",Egyptian Social Structure,"Egyptian society was structured like a pyramid. At the top were the gods, such as Ra, Osiris, and Isis. Egyptians believed that the gods controlled the universe. Therefore, it was important to keep them happy. They could make the Nile overflow, cause famine, or even bring death. \r\nThe Egyptians also elevated some human beings to gods. Their leaders, called pharaohs, were believed to be gods in human form. They had absolute power over their subjects. After pharaohs died, huge stone pyramids were built as their tombs. Pharaohs were buried in chambers within the pyramids. \r\nBecause the pe..."
2,814d6b,Summarize how the Third Wave developed over such a short period of time and why the experiment was ended.,The Third Wave,"Background \r\nThe Third Wave experiment took place at Cubberley High School in Palo Alto, California during the first week of April 1967. History teacher Ron Jones, finding himself unable to explain to his students how people throughout history followed the crowd even when terrible things were happening, decided to demonstrate it to his students through an experiment. Jones announced that he was starting a movement aimed to eliminate democracy. Jones named the movement “The Third Wave” as a symbol of strength, referring to the mythical belief that the third in a series of waves is the str..."
3,ebad26,Summarize the various ways the factory would use or cover up spoiled meat. Cite evidence in your answer.,Excerpt from The Jungle,"With one member trimming beef in a cannery, and another working in a sausage factory, the family had a first-hand knowledge of the great majority of Packingtown swindles. For it was the custom, as they found, whenever meat was so spoiled that it could not be used for anything else, either to can it or else to chop it up into sausage. With what had been told them by Jonas, who had worked in the pickle rooms, they could now study the whole of the spoiled-meat industry on the inside, and read a new and grim meaning into that old Packingtown jest—that they use everything of the pig except the ..."


In [21]:
df_test.head()

,prompt_id,prompt_question,prompt_title,prompt_text
0,abc123,Summarize...,Example Title 1,Heading\nText...
1,def789,Summarize...,Example Title 2,Heading\nText...
